In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

In [2]:
test_df=pd.read_csv('test.csv')

In [3]:
from sklearn.impute import SimpleImputer
si_mode=SimpleImputer(strategy='most_frequent')
si_mean=SimpleImputer(strategy='mean')

In [4]:
df1=test_df.copy()

In [5]:
df1['HomePlanet']=si_mode.fit_transform(df1[['HomePlanet']]).ravel()

In [6]:
df1['RoomService']=si_mode.fit_transform(df1[['RoomService']]).ravel()

In [7]:
df1['FoodCourt']=si_mode.fit_transform(df1[['FoodCourt']]).ravel()
df1['ShoppingMall']=si_mode.fit_transform(df1[['ShoppingMall']]).ravel()
df1['Spa']=si_mode.fit_transform(df1[['Spa']]).ravel()
df1['VRDeck']=si_mode.fit_transform(df1[['VRDeck']]).ravel()

In [8]:
df1.loc[(df1['RoomService']==0.0) 
        & (df1['FoodCourt']==0.0) 
        & (df1['ShoppingMall']==0.0) 
        & (df1['Spa']==0.0) 
        & (df1['VRDeck']==0.0) 
        & (df1['CryoSleep'].isna()==True),'CryoSleep']=True

In [9]:
df1['Age']=si_mean.fit_transform(df1[['Age']]).ravel()

In [10]:
df1.loc[(df1['RoomService']!=0.0) 
        & (df1['FoodCourt']!=0.0) 
        & (df1['ShoppingMall']!=0.0) 
        & (df1['Spa']!=0.0) 
        & (df1['VRDeck']!=0.0) 
        & (df1['VIP'].isna()==True)
        &(df1['Age']>20),'VIP']=True

In [11]:
df1['Destination']=si_mode.fit_transform(df1[['Destination']]).ravel()

In [12]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
oe=OneHotEncoder()
le=LabelEncoder()

In [13]:
oe.fit(df1[['HomePlanet']])
encoded_cols_home_planet=oe.get_feature_names_out()
print(encoded_cols_home_planet)

['HomePlanet_Earth' 'HomePlanet_Europa' 'HomePlanet_Mars']


In [14]:
df1[encoded_cols_home_planet]=oe.transform(df1[['HomePlanet']]).toarray()

In [15]:
le_cols=['CryoSleep','VIP']
for i in range(len(le_cols)):
    df1[le_cols[i]]=le.fit_transform(df1[le_cols[i]])

In [16]:
oe.fit(df1[['Destination']])
encoded_cols_destination=oe.get_feature_names_out()
print(encoded_cols_destination)

['Destination_55 Cancri e' 'Destination_PSO J318.5-22'
 'Destination_TRAPPIST-1e']


In [17]:
df1[encoded_cols_destination]=oe.transform(df1[['Destination']]).toarray()

In [18]:
final_cols=list(df1.drop(columns=['PassengerId','Cabin','Destination','Name','HomePlanet']).columns)
fin_df=df1[final_cols]
fin_df.head()

,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
0,1,27.0,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1,0,19.0,0,0.0,9.0,0.0,2823.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
2,1,31.0,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
3,0,38.0,0,0.0,6652.0,0.0,181.0,585.0,0.0,1.0,0.0,0.0,0.0,1.0
4,0,20.0,0,10.0,0.0,635.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


In [19]:
train_df=pd.read_csv('cleaned_df.csv')

In [20]:
fin_df=train_df.copy()

In [21]:
input_cols=list(fin_df.drop(columns=['Name','Transported','Unnamed: 0','PassengerId','HomePlanet','Cabin','Destination']))
op_col='Transported'
print(input_cols)

['CryoSleep', 'Age', 'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'HomePlanet_Earth', 'HomePlanet_Europa', 'HomePlanet_Mars', 'Destination_55 Cancri e', 'Destination_PSO J318.5-22', 'Destination_TRAPPIST-1e']


In [22]:
x=fin_df[input_cols]
y=fin_df[op_col]

In [23]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [24]:
x_train.columns

Index(['CryoSleep', 'Age', 'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall',
       'Spa', 'VRDeck', 'HomePlanet_Earth', 'HomePlanet_Europa',
       'HomePlanet_Mars', 'Destination_55 Cancri e',
       'Destination_PSO J318.5-22', 'Destination_TRAPPIST-1e'],
      dtype='object')

In [25]:
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier(max_depth=10,n_estimators=300,max_features='sqrt')
rf.fit(x_train,y_train)

,n_estimators,300
,criterion,'gini'
,max_depth,10
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [26]:
rf.score(x_test,y_test)

0.7906843013225991

In [27]:
df1['PassengerId']

0       0013_01
1       0018_01
2       0019_01
3       0021_01
4       0023_01
         ...   
4272    9266_02
4273    9269_01
4274    9271_01
4275    9273_01
4276    9277_01
Name: PassengerId, Length: 4277, dtype: object

In [28]:
y_pred=rf.predict(df1[input_cols])

In [29]:
y_pred.ravel()

array([1, 0, 1, ..., 1, 1, 0], shape=(4277,))

In [30]:
dic={'PassengerId':df1.PassengerId,'Transported':y_pred.ravel()}

In [31]:
map_={0:False,1:True}

In [32]:
submission_df=pd.DataFrame(dic)
submission_df['Transported']=submission_df['Transported'].map(map_)
submission_df

,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,True
...,...,...
4272,9266_02,True
4273,9269_01,False
4274,9271_01,True
4275,9273_01,True


In [35]:
submission_df.to_csv('submission_df.csv',index=False)

In [34]:
?RandomForestClassifier

Init signature:
RandomForestClassifier(
    n_estimators=100,
    *,
    criterion='gini',
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=1,
    min_weight_fraction_leaf=0.0,
    max_features='sqrt',
    max_leaf_nodes=None,
    min_impurity_decrease=0.0,
    bootstrap=True,
    oob_score=False,
    n_jobs=None,
    random_state=None,
    verbose=0,
    warm_start=False,
    class_weight=None,
    ccp_alpha=0.0,
    max_samples=None,
    monotonic_cst=None,
)
Docstring:     
A random forest classifier.

A random forest is a meta estimator that fits a number of decision tree
classifiers on various sub-samples of the dataset and uses averaging to
improve the predictive accuracy and control over-fitting.
Trees in the forest use the best split strategy, i.e. equivalent to passing
`splitter="best"` to the underlying :class:`~sklearn.tree.DecisionTreeClassifier`.
The sub-sample size is controlled with the `max_samples` parameter if
`bootstrap=True` (default), otherwise the